# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to sdas26@uic.edu and will expire on September 26, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Sanjib\AppData\Local\Temp\graphlab_server_1481733358.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10, verbose= False)

Find what features had non-zero weight.

In [74]:
model_all["coefficients"][model_all["coefficients"]["value"]>0].print_rows(num_rows=20)


+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[6 rows x 4 columns]



Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [19]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [26]:
import numpy as np
l1_penalty_values = np.logspace(1,7, num=13)
val_err= {}

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training,target = 'price',features = all_features, validation_set=None,
                                             l1_penalty=l1_penalty, l2_penalty=0., verbose=False)
    predictions= model.predict(validation)
    residual = validation['price'] - predictions
    rss = (residual*residual).sum()
    val_err[l1_penalty]=rss


(10.0, 625766285142459.9)

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [ ]:
min(val_err.items(),key=lambda x: x[1])

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [35]:
model_test = graphlab.linear_regression.create(training,target='price', features=all_features,verbose=False,
                                              validation_set=None,l1_penalty=10, l2_penalty=0.)

print (model_test["coefficients"][model_test["coefficients"]["value"]>0]).print_rows(num_rows=20)
print model_test.coefficients["value"].nnz()
#predictions= model_test.predict(testing)
#residual= testing['price']- predictions
#rss = (residual*residual).sum()

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [36]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [37]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [38]:
penalty_nz_list={}
for l1_penalty in l1_penalty_values:
    model_nz = graphlab.linear_regression.create(training,target='price', features=all_features,verbose=False,
                                              validation_set=None,l1_penalty=l1_penalty, l2_penalty=0.)
    penalty_nz_list[l1_penalty]= model_nz.coefficients["value"].nnz()

In [48]:
penalty_nz_list

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313128: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [57]:
l1_penalty_min = max((dict((k,v) for k,v in penalty_nz_list.items() if v>max_nonzeros)).keys())
l1_penalty_max = min((dict((k,v) for k,v in penalty_nz_list.items() if v<max_nonzeros)).keys())

In [58]:
print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [77]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [78]:
rss_penalty_nz={}
for l1_penalty in l1_penalty_values:
    model_nr = graphlab.linear_regression.create(training,target='price', features=all_features,verbose=False,
                                              validation_set=None,l1_penalty=l1_penalty, l2_penalty=0.)
    predictions=model_nr.predict(validation)
    residual = validation['price']-predictions
    rss = (residual*residual).sum()    
    rss_penalty_nz[l1_penalty]= rss, model_nr.coefficients["value"].nnz()
    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [79]:
rss_penalty_nz

{2976351441.6313128: (966925692362084.5, 10),
 3019316638.952415: (974019450084556.1, 10),
 3062281836.2735171: (981188367942452.8, 10),
 3105247033.5946193: (989328342459474.0, 10),
 3148212230.9157214: (998783211265891.2, 10),
 3191177428.2368236: (1008477167020094.0, 10),
 3234142625.5579257: (1018298780553819.8, 10),
 3277107822.8790278: (1028247992205977.2, 10),
 3320073020.20013: (1034616909232828.1, 8),
 3363038217.5212321: (1038554735941040.8, 8),
 3406003414.8423343: (1043237237871703.0, 8),
 3448968612.1634364: (1046937488751711.1, 7),
 3491933809.4845386: (1051147625612860.9, 7),
 3534899006.8056407: (1055992735342999.1, 7),
 3577864204.1267428: (1060799531763287.8, 7),
 3620829401.447845: (1065707689498230.1, 6),
 3663794598.7689471: (1069464335425586.5, 6),
 3706759796.0900493: (1073504549585599.6, 6),
 3749724993.4111514: (1077632775581416.0, 6),
 3792690190.7322536: (1081867592324110.6, 6)}

In [80]:

for k,v in rss_penalty_nz.iteritems():
    if(v[1]==max_nonzeros) and (v[0]<1060799531763287.8): # this section is dynamic enough and it is hard coded
        lowestRSS = v[0]
        bestpenalty = k
print bestpenalty, lowestRSS

3534899006.81 1.05599273534e+15


In [81]:
model_best = graphlab.linear_regression.create(training,target='price', features=all_features,verbose=False,
                                              validation_set=None,l1_penalty=bestpenalty, l2_penalty=0.)

In [82]:
model_best["coefficients"][model_best["coefficients"]["value"] >0]

name,index,value,stderr
(intercept),None,226807.047234,None
bedrooms,None,322.098888436,None
bathrooms,None,15392.9565223,None
sqft_living,None,31.9817554069,None
sqft_living_sqrt,None,666.949904077,None
grade,None,2747.52787565,None
sqft_above,None,29.4061078857,None
